http://stackoverflow.com/questions/37508659/group-by-and-count-distinct-words-in-pandas-dataframe

http://stackoverflow.com/questions/14262433/large-data-work-flows-using-pandas


In [1]:
!ls -Lilah ./share/Data
import pandas as pd
import numpy as np
import plotly.plotly as py
import re



ls: Zugriff auf './share/Data' nicht möglich: Datei oder Verzeichnis nicht gefunden


In [2]:
# Stopwords are downloaded and defined here
try:
    with open("german_stopwords_full.txt") as f:
        STOPWORDS = [line.strip() for line in f if not line.startswith(";")]
except FileNotFoundError:
    !wget https://raw.githubusercontent.com/solariz/german_stopwords/master/german_stopwords_full.txt
    with open("german_stopwords_full.txt") as f:
        STOPWORDS = [line.strip() for line in f if not line.startswith(";")] 
      

SW = ["dass"]
_ = [STOPWORDS.append(w) for w in SW]

In [3]:
# generate the raw_data from CSV
raw_data = pd.read_csv("../share/Data/SPON_complete", delimiter=",", skipinitialspace=True)

In [4]:
def count_words(source):
    """ Counting the words of the column article of a given Dataframe.
    
    It is possible to define a word, so only this word will be counted.
    """
    
    #split column Message to new df, create Serie by stack
    s = (source.article.str.split(expand=True).stack().str.lower()   )
    #remove multiindex
    s.index = s.index.droplevel(-1)
    s.name= 'words'
    #join Serie s to df source
    df = (source.join(s))
    
    # Cleaning the strings to be only alphanumeric
    df['words'] = df['words'].map(lambda x: re.sub(r'\W+', '', str(x)))
    
    df = df[~df['words'].isin(STOPWORDS)].groupby(['words']).size().reset_index(name='count').sort_values(by='count')
    
    return df


In [5]:
def wordcounter(source, word):
    df = count_words(source)
    return df[df['words'].str.contains(word)]

In [6]:
# This will generate a list of Strings, representating the months from .01.2001 to .12.2012
datestrings = ( ".{:02d}.20{:02d}".format(m,y)  for y in range(1, 17) for m in range(1, 13) )

In [7]:
# Count the absolute existence of one word
absolute_count = lambda d, w : wordcounter(raw_data[raw_data.day.str.contains(d, na=False)],
           word=w)['count'].sum()

In [ ]:
## TODO:
#   - Tabbelenüberschrift mit gesuchtem Wort, sonst "words"
#   - konkatenieren von verschiedenen Suchbegriffen in einer 
#     großen Tabelle
#   - Schreiben der Daten in persistente Datenstruktur (CSV)

In [ ]:

# Testdaten

A = ['.01.2016', '.02.2016', '.03.2016', '.04.2016', '.05.2016', '.06.2016',
     '.07.2016', '.08.2016', '.09.2016', '.10.2016', '.11.2016', '.12.2016']

t =[absolute_count(a, 'pegida') for a in A]

m = (   "Jan", "Feb", "Mär", "Apr", "Mai", "Jun", 
        "Jul", "Aug", "Sep", "Okt", "Nov", "Dez")
d = pd.DataFrame(t, index=m)
d.columns = ["paket"]

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

d.plot(
    kind='bar', 
    title='Anzahl der Worte {} nach Monaten'.format(d.columns.values))
